In [79]:
import os
import json
import copy
import pandas as pd
import math
import seaborn as sns


In [80]:
def flatten_dict(d, prefix):
    is_dict = isinstance(d, dict)

    newdict = dict()
    if is_dict:
        for k, v in d.items():
            nk = f"{prefix}.{k}"
            newdict[nk] = copy.deepcopy(v)
    return newdict


def flatten_list(d, prefix):
    is_list = isinstance(d, list)

    newdict = dict()
    if is_list:
        r = []
        for i, v in enumerate(d):
            nk = f"{prefix}[{i}]"
            r.append(flatten_dict(v, nk))
        for dic in r:
            newdict = newdict | dic

    return newdict

In [81]:
FOLDER = "./results"
PREFIX = "esvis-ds"

folders_with_call_file = []

for subdir, dirs, files in os.walk(FOLDER):
    if "call.json" in files and subdir.startswith(os.path.join(FOLDER, PREFIX)):
        folders_with_call_file.append(subdir)

calls = []
for dir in folders_with_call_file:
    callfile = os.path.join(dir, "call.json")
    with open(callfile, "r") as f:
        call = json.load(f)
        call = call | flatten_dict(call["ctx"], "ctx")
        del call["ctx"]
        if "metrics" in call:
            call = call | flatten_list(call["metrics"], "metrics")
            del call["metrics"]

        # merge also the additional files
        for ilp_status_file in ["layout", "overlapremoval", "route"]:
            filename = f"ilp_status_{ilp_status_file}.json"
            filepath = os.path.join(dir, filename)
            status_file_exists = os.path.exists(filepath)
            if status_file_exists:
                with open(filepath, "r") as f:
                    status_file_data = flatten_dict(
                        json.load(f), f"ilp_{ilp_status_file}"
                    )
                    call = call | status_file_data

        calls.append(call)

df = pd.DataFrame(data=calls)
df

,success,duration_ms,ctx.general.strategy,ctx.general.gridwidth,ctx.general.gridheight,ctx.general.gridpad,ctx.general.readdir,ctx.general.writedir,ctx.general.numlayers,ctx.layout.overlapremover,...,ilp_route.mip,traceback,ilp_route.has_mipgap,ilp_route.mipgap,ilp_route.runtime_ms,ilp_route.status,ilp_overlapremoval.has_mipgap,ilp_overlapremoval.mipgap,ilp_overlapremoval.runtime_ms,ilp_overlapremoval.status
0,True,7.032794e+04,heuristic,16,16,1,/home1/npiccolotto/ensemble-sets/code/designsp...,/tmp/3432583.1.all.q,3,dgrid,...,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,False,7.907380e+05,heuristic,32,32,1,/home1/npiccolotto/ensemble-sets/code/designsp...,/tmp/3432579.1.all.q,3,dgrid,...,True,"Traceback (most recent call last):\n File ""/h...",True,inf,1.288699e+04,3.0,NaN,NaN,NaN,NaN
2,False,3.797054e+06,heuristic,16,16,1,/home1/npiccolotto/ensemble-sets/code/designsp...,/tmp/3432612.1.all.q,3,dgrid,...,NaN,"Traceback (most recent call last):\n File ""/h...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,False,7.722211e+05,heuristic,32,32,1,/home1/npiccolotto/ensemble-sets/code/designsp...,/tmp/3432578.1.all.q,3,dgrid,...,NaN,"Traceback (most recent call last):\n File ""/h...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,False,2.877553e+04,heuristic,32,32,1,/home1/npiccolotto/ensemble-sets/code/designsp...,/tmp/3432587.1.all.q,3,dgrid,...,NaN,"Traceback (most recent call last):\n File ""/h...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,True,6.223837e+04,heuristic,16,16,1,/home1/npiccolotto/ensemble-sets/code/designsp...,/tmp/3432582.1.all.q,3,dgrid,...,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,False,3.666769e+06,heuristic,16,16,1,/home1/npiccolotto/ensemble-sets/code/designsp...,/tmp/3432574.1.all.q,3,dgrid,...,True,"Traceback (most recent call last):\n File ""/h...",True,inf,3.600161e+06,9.0,NaN,NaN,NaN,NaN
7,False,2.942276e+04,heuristic,32,32,1,/home1/npiccolotto/ensemble-sets/code/designsp...,/tmp/3432589.1.all.q,3,dgrid,...,NaN,"Traceback (most recent call last):\n File ""/h...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,False,7.016396e+05,heuristic,32,32,1,/home1/npiccolotto/ensemble-sets/code/designsp...,/tmp/3432575.1.all.q,3,dgrid,...,NaN,"Traceback (most recent call last):\n File ""/h...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,True,6.336810e+04,heuristic,16,16,1,/home1/npiccolotto/ensemble-sets/code/designsp...,/tmp/3432602.1.all.q,3,hagrid,...,False,NaN,NaN,NaN,NaN,NaN,True,0.0,2187.755108,2.0


In [75]:
# post process

df["success"] = df.success.astype("bool")


def sum_(l):
    return sum([i for i in l if not math.isnan(i)])


# aggregate metrics over layers
metrics = [
    "total_line_crossings",
    "total_edge_crossings",
    "total_lines",
    "total_edges",
    "total_node_moves",
]
for metric in metrics:
    df[metric] = sum_([df[f"metrics[{i}].{metric}"] for i in range(3)])

#

In [94]:
df[['ilp_route.status','ctx.route.router','ctx.route.subsupporttype','ctx.route.subsupportgrouping']]

,ilp_route.status,ctx.route.router,ctx.route.subsupporttype,ctx.route.subsupportgrouping
0,NaN,heuristic,steiner-tree,intersection-group
1,3.0,opt,steiner-tree,intersection-group
2,NaN,opt,steiner-tree,intersection-group
3,NaN,opt,steiner-tree,intersection-group
4,NaN,heuristic,steiner-tree,intersection-group
5,NaN,heuristic,steiner-tree,intersection-group
6,9.0,opt,steiner-tree,intersection-group
7,NaN,heuristic,steiner-tree,intersection-group
8,NaN,opt,steiner-tree,intersection-group
9,NaN,heuristic,steiner-tree,intersection-group


In [92]:
# sns.histplot(df, x='success', bins=2, hue='success')

sns.violinplot(df, y="total_lines", x="ctx.general.gridwidth")

ValueError: Could not interpret input 'total_lines'